In [1]:
import json
import os

In [2]:
def get_directories(path):
    dirs = [dir for dir in os.listdir(path) if os.path.isdir(os.path.join(path, dir))]
    return dirs

In [3]:
def load_jsonl(path):
    lines = []
    with open(path, "r") as file:
        for line in file:
            lines.append(json.loads(line))
    return lines

In [4]:
def get_model_dir(model_dirs, gen_mode, model_name_escaped, dedup_str, tool_call):
    for model_dir in model_dirs:
        if gen_mode == "structured" and "unstructured" in model_dir:
            continue

        if gen_mode in model_dir and model_name_escaped in model_dir and dedup_str in model_dir and tool_call in model_dir:
            return model_dir

In [45]:
def index_it(idx, questions, solutions, generations, scores, idx_type="error"):

    if idx_type == "absolute":
        question = questions[idx]
        solution = solutions[idx]
        generation = generations[idx]
        score = [score for i, score in enumerate(scores) if i > 0 and score["id"] == idx]
    elif idx_type == "error":
        score = scores[idx+1]
        abs_idx = score["id"] - 1
        question = questions[abs_idx]
        solution = solutions[abs_idx]
        generation = generations[abs_idx]
    else:
        raise ValueError("Wrong idx_type.")

    return question, solution, generation, score

In [5]:
# Inputs
bfcl_category = "simple"

dedup = True
gen_mode = "structured"
tool_call = "auto"
model_name = "databricks/dbrx-instruct"
out_dir = "./outputs"

In [15]:
# Get paths
test_file = f"gorilla_openfunctions_v1_test_{bfcl_category}.json"
questions_path = os.path.join("./data", test_file)
solutions_path = os.path.join("./data/possible_answer", test_file)


model_name_escaped = model_name.replace("/", "_")
dedup_str = "dedup" if dedup else ""
model_dirs = get_directories(out_dir)
model_dir = get_model_dir(model_dirs, gen_mode, model_name_escaped, dedup_str, tool_call)
generations_path = os.path.join(out_dir, model_dir, "generations", test_file)
scores_path = os.path.join(out_dir, model_dir, "scores", bfcl_category + "_score.json")

In [17]:
# Load all files
questions = load_jsonl(questions_path)
solutions = load_jsonl(solutions_path)
generations = load_jsonl(generations_path)
scores = load_jsonl(scores_path)

In [46]:
idx = 0
idx_type = "error"

values = index_it(idx, questions, solutions, generations, scores, idx_type="error")
question, solution, generation, score = *values

SyntaxError: can't use starred expression here (2884515939.py, line 5)

In [43]:
question

{'question': 'Find the area of a triangle with a base of 10 units and height of 5 units.',
 'function': {'name': 'calculate_triangle_area',
  'description': 'Calculate the area of a triangle given its base and height.',
  'parameters': {'type': 'dict',
   'properties': {'base': {'type': 'integer',
     'description': 'The base of the triangle.'},
    'height': {'type': 'integer',
     'description': 'The height of the triangle.'},
    'unit': {'type': 'string',
     'description': "The unit of measure (defaults to 'units' if not specified)"}},
   'required': ['base', 'height']}}}

In [42]:
score

{'id': 1,
 'model_name': 'databricks/dbrx-instruct',
 'test_category': 'simple',
 'valid': False,
 'error': ['Wrong number of functions.'],
 'error_type': 'simple_function_checker:wrong_count',
 'prompt': {'question': 'Find the area of a triangle with a base of 10 units and height of 5 units.',
  'function': {'name': 'calculate_triangle_area',
   'description': 'Calculate the area of a triangle given its base and height.',
   'parameters': {'type': 'dict',
    'properties': {'base': {'type': 'integer',
      'description': 'The base of the triangle.'},
     'height': {'type': 'integer',
      'description': 'The height of the triangle.'},
     'unit': {'type': 'string',
      'description': "The unit of measure (defaults to 'units' if not specified)"}},
    'required': ['base', 'height']}}},
 'model_result_raw': "[calculate_triangle_area(base=10, height=5), calculate_triangle_area(base=10, height=5, unit='units')]",
 'model_result_decoded': [{'calculate_triangle_area': {'base': 10,
   